In [1]:
import duckdb
from choose_estacoes.pickers import haversine
import pandas as pd


conn_prata = duckdb.connect('r2_prata.duckdb')
def sql(query):
    return conn_prata.execute(query).fetch_df()

In [2]:
conn_ouro = duckdb.connect('r2_ouro.duckdb')

In [4]:
dim_estacoes = sql("SELECT DISTINCT id_estacao,lat,lon FROM dim_estacoes")

In [5]:
combinacoes = pd.merge(dim_estacoes, dim_estacoes, how='cross', suffixes=('_1', '_2'))
combinacoes = combinacoes.loc[combinacoes['id_estacao_1'] != combinacoes['id_estacao_2']]
combinacoes['distancia_km'] = haversine(
    combinacoes['lat_1'], combinacoes['lon_1'],
    combinacoes['lat_2'], combinacoes['lon_2']
)

In [6]:
conn_ouro = duckdb.connect('r2_ouro.duckdb')

In [7]:
conn_ouro.execute("CREATE TABLE IF NOT EXISTS 'estacoes_distancias' AS SELECT * FROM combinacoes")